<a href="https://colab.research.google.com/github/samjurassic/datascience-demo/blob/main/recycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import pandas as pd
import seaborn as sns
import datetime as dt


In [ ]:
recycle = pd.read_csv("/content/Recycling_Diversion_and_Capture_Rates_20240608.csv")

In [ ]:
recycle.head()

In [ ]:
def col_formatter(x):
  xs = re.split(r"\({1,2}", x)[0].lower().strip()
  xs = re.sub("\s|-", "_", xs)
  return xs

recycle.columns = [col_formatter(c) for c in recycle.columns]

In [ ]:
recycle.describe(include="all")

In [ ]:
def date_formatter(yr, mo):
  d = dt.datetime.strptime(f"{yr}-{mo}", "%Y-%B").date()
  return d

In [ ]:
recycle["date"] = recycle.apply(lambda df: date_formatter(df.fiscal_year, df.month_name), axis=1)

recycle["calendar_month"] = recycle.date.apply(lambda x: x.month)

In [ ]:
recycle.head()

In [ ]:
rzm = (recycle
       .groupby(["zone", "month_name", "calendar_month"])["diversion_rate_total"]
       .mean()
       .reset_index()
       .sort_values(["zone", "calendar_month"]))

In [ ]:
rzm.head()

In [ ]:
def month_name(x):
  return dt.date(2020, x, 1).strftime("%B")

In [ ]:
df = rzm.pivot(index="zone", columns=["calendar_month"], values="diversion_rate_total")

sns.heatmap(df, xticklabels=[month_name(c) for c in df.columns.to_list()], cmap="Greens")

In [ ]:
g = sns.FacetGrid(recycle, col="zone", col_wrap=3)
g.map_dataframe(sns.histplot, x="capture_rate_paper")


# sns.histplot(recycle, x="capture_rate_paper", hue="zone")

In [ ]:
rzm = (recycle
       .groupby(["fiscal_year", "zone"])["diversion_rate_total"]
       .mean()
       .reset_index()
       .sort_values(["fiscal_year", "zone"]))

In [ ]:
sns.lineplot(rzm, x="fiscal_year", y="diversion_rate_total", hue="zone")

**Assignment:**

1. Make a lineplot of mean capture_rate_paper from 2016-2019 grouped by zone and fiscal_year
2. What district had the highest average diversion_rate_total in each fiscal_year?
3. What district had the lowest average diversion_rate_total in each fiscal_year?
4. Make a barplot of annual mean capture_rate_paper for Brooklyn South and Brooklyn North combined into one "Brooklyn"
5. Extra credit: make a plot of your choice